audio generation

In [1]:
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech()

import os
from google.colab import files

fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/tortoise-tts
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post2-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.9.2-py3-none-any.whl.metadata (8.2 kB)
  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of resampy to determine wh

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:392: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

100% (1716988501 of 1716988501) |########| Elapsed Time: 0:00:24 Time:  0:00:24


Done.


100% (60938957 of 60938957) |############| Elapsed Time: 0:00:00 Time:  0:00:00


...

100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:02 Time:  0:00:02


Done.


/content/tortoise-tts/tortoise/api.py:196: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict(torch.load(f'{models_dir}/autoregressive.pth'))


In [2]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"


In [4]:
import torchaudio
import IPython.display
import os
import csv

In [5]:
text_file_path = '/content/drive/MyDrive/mini 2 test result/Hidden_generations/split_transcriptions_testing.txt'

for M2


In [6]:
import os
import shutil
import re
from google.colab import drive

# # Mount Google Drive if using Colab
# drive.mount('/content/drive')

# Define the custom voice folder path and input directory path
CUSTOM_VOICE_NAME = "M2_50_sample"
custom_voice_folder = f"/content/tortoise-tts/tortoise/voices/{CUSTOM_VOICE_NAME}"
input_dir = '/content/drive/MyDrive/mini 2 test result/Denoised audios/M2_test_results'  # Replace with your actual input directory path

# Ensure the input directory exists
if not os.path.exists(input_dir):
    print(f"Input directory '{input_dir}' does not exist.")
else:
    # Ensure the custom voice folder exists
    os.makedirs(custom_voice_folder, exist_ok=True)

    # Function to extract numbers from filenames for natural sorting
    def natural_sort_key(filename):
        return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', filename)]

    # List all WAV files in the input directory and sort naturally
    audio_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.wav')], key=natural_sort_key)

    # Copy each file from the input directory to the custom voice folder
    for i, audio_file in enumerate(audio_files):
        src_path = os.path.join(input_dir, audio_file)
        dest_path = os.path.join(custom_voice_folder, f'{i}.wav')

        shutil.copy(src_path, dest_path)
        print(f'Copied {audio_file} to {dest_path}')

    if not audio_files:
        print("No WAV files found in the input directory.")


Copied M2_mixed_denoise_1.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/0.wav
Copied M2_mixed_denoise_2.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/1.wav
Copied M2_mixed_denoise_3.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/2.wav
Copied M2_mixed_denoise_4.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/3.wav
Copied M2_mixed_denoise_5.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/4.wav
Copied M2_mixed_denoise_6.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/5.wav
Copied M2_mixed_denoise_7.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/6.wav
Copied M2_mixed_denoise_8.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/7.wav
Copied M2_mixed_denoise_9.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/8.wav
Copied M2_mixed_denoise_10.wav to /content/tortoise-tts/tortoise/voices/M2_50_sample/9.wav


In [10]:
# Define paths
# Path to your text file
output_dir = '/content/drive/MyDrive/mini 2 test result/Hidden_generations/M2_hidden_gen'  # Path to your output directory
#csv_file_path = '/content/drive/MyDrive/mini 2 test result/Hidden_generations/M2_hidden_gen/'  # Path for the CSV file

In [13]:
csv_file_path = '/content/drive/MyDrive/mini 2 test result/Hidden_generations/M2_hidden_gen/details.csv'
# Load texts from the file
with open(text_file_path, 'r') as file:
    texts = file.readlines()

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write header row
    csv_writer.writerow(['Audio File Name', 'Text'])

    # Generate audio for each text
    for idx, text in enumerate(texts, start=1):
        text = text.strip()  # Remove any leading/trailing whitespace

        # Load voice samples and conditioning latents
        voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

        # Generate speech
        gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)

        # Save the generated audio
        output_file_name = f'M1_output_{idx}.wav'
        output_path = os.path.join(output_dir, output_file_name)
        torchaudio.save(output_path, gen.squeeze(0).cpu(), 24000)

        # Write the audio file name and text to the CSV file
        csv_writer.writerow([output_file_name, text])

        print(f'Generated and saved audio for text {idx} to {output_file_name}')

        # Optionally, display the audio (comment out if not needed)
        # IPython.display.Audio(output_path)

/content/tortoise-tts/tortoise/models/arch_util.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch.load(self.mel_norm_file)


Generating autoregressive samples..


100%|██████████| 16/16 [00:32<00:00,  2.03s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 16/16 [00:43<00:00,  2.74s/it]


...

Generated and saved audio for text 9 to M1_output_9.wav
Generating autoregressive samples..


100%|██████████| 16/16 [00:48<00:00,  3.01s/it]


Computing best candidates using CLVP and CVVP


 12%|█▎        | 2/16 [00:07<00:52,  3.78s/it]


KeyboardInterrupt: 

In [ ]:
!pip install openpyxl
import openpyxl

# Load the workbook
workbook = openpyxl.load_workbook(text_file_path)

# Select the active worksheet
worksheet = workbook.active

# Extract text from the first column
texts = [cell.value for row in worksheet.iter_rows() for cell in row]

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write header row
    csv_writer.writerow(['Audio File Name', 'Text'])

    # Generate audio for each text
    for idx, text in enumerate(texts, start=1):
        text = text.strip()  # Remove any leading/trailing whitespace
        if text: # check if text exists
          # Load voice samples and conditioning latents
          voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

          # Generate speech
          gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)

          # Save the generated audio
          output_file_name = f'M1_output_{idx}.wav'
          output_path = os.path.join(output_dir, output_file_name)
          torchaudio.save(output_path, gen.squeeze(0).cpu(), 24000)

          # Write the audio file name and text to the CSV file
          csv_writer.writerow([output_file_name, text])

          print(f'Generated and saved audio for text {idx} to {output_file_name}')

/content/tortoise-tts/tortoise/models/arch_util.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch.load(self.mel_norm_file)


Generating autoregressive samples..


100%|██████████| 16/16 [01:44<00:00,  6.53s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 16/16 [00:44<00:00,  2.80s/it]


...

Generated and saved audio for text 2 to M1_output_2.wav
Generating autoregressive samples..


100%|██████████| 16/16 [01:27<00:00,  5.46s/it]


Computing best candidates using CLVP and CVVP


 44%|████▍     | 7/16 [00:21<00:27,  3.03s/it]


KeyboardInterrupt: 